# Generación de audio TTS desde PDF en Supabase (fin a fin)

Este cuaderno: (1) descarga el PDF del libro desde Supabase, (2) extrae TODO el texto con PyPDF2 y fallback pdfminer.six, (3) sintetiza audio con Microsoft Edge TTS por fragmentos mostrando progreso, (4) guarda un .txt junto al PDF y sube el MP3 a audios_libros/{id_libro}/{short_name}.mp3, y (5) actualiza tbl_libros_x_usuarios (audio, id_voz, id_playbackrate).

In [ ]:
# Instalación/aseguramiento de paquetes requeridos
import sys, subprocess, importlib
def ensure(mod, pkg):
    try:
        importlib.import_module(mod)
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--user', pkg])
for mod, pkg in [
    ('dotenv', 'python-dotenv'),
    ('pdfminer.high_level', 'pdfminer.six'),
    ('PyPDF2', 'PyPDF2'),
    ('edge_tts', 'edge-tts'),
    ('supabase', 'supabase'),
    ('requests', 'requests'),
]:
    ensure(mod, pkg)
print('✅ Paquetes verificados')

In [ ]:
# Configuración básica
import os, re, asyncio
from datetime import datetime
from io import BytesIO
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from pdfminer.high_level import extract_text as pdf_extract_text
import edge_tts, requests
from supabase import create_client

# Parámetros (ajusta si lo deseas)
BOOK_ID = 1
USER_ID = 1
VOICE_ID = 1  # 1..16
RATE = 1.0    # 1.0 normal; 1.1 = +10%
MAX_CHARS = 3000  # tamaño prudente de fragmentos para edge-tts

# Cargar backend/.env
env_path = os.path.join(os.path.dirname(__file__) if '__file__' in globals() else '.', '..', '.env')
if os.path.exists(env_path):
    load_dotenv(env_path, override=True)
print('✅ Entorno cargado')

SUPABASE_URL = os.environ.get('SUPABASE_URL', '').strip()
SUPABASE_SERVICE_ROLE_KEY = os.environ.get('SUPABASE_SERVICE_ROLE_KEY', '').strip()
if not SUPABASE_URL or not SUPABASE_SERVICE_ROLE_KEY:
    raise RuntimeError('Faltan SUPABASE_URL o SUPABASE_SERVICE_ROLE_KEY en .env')
print('🔑 Supabase URL ok')

supabase = create_client(SUPABASE_URL, SUPABASE_SERVICE_ROLE_KEY)
BUCKET_ARCHIVOS = os.environ.get('BUCKET_ARCHIVOS', 'archivos_libros')
BUCKET_AUDIOS = os.environ.get('BUCKET_AUDIOS', 'audios_libros')
print('✅ Cliente Supabase listo')

In [ ]:
# Utilidades Supabase y helpers
from urllib.parse import urlparse

def resolve_bucket_and_path_from_url(public_url: str):
    try:
        u = urlparse(public_url)
        idx = u.path.find('/storage/v1/object/public/')
        if idx != -1:
            rest = u.path[idx + len('/storage/v1/object/public/'):]
            parts = rest.split('/')
            bucket = parts[0]
            path = '/'.join(parts[1:])
            return bucket, path
    except Exception:
        pass
    return None, None

def get_voice_short_name(voice_id: int) -> str:
    res = supabase.table('tbl_voces').select('id_voz, short_name').eq('id_voz', voice_id).gte('id_voz', 1).lte('id_voz', 16).maybe_single().execute()
    item = getattr(res, 'data', None)
    if not item:
        raise RuntimeError('Voz no encontrada o fuera de rango (1..16)')
    return item['short_name']

def get_book(book_id: int):
    res = supabase.table('tbl_libros').select('id_libro, titulo, archivo').eq('id_libro', book_id).maybe_single().execute()
    item = getattr(res, 'data', None)
    if not item:
        raise RuntimeError('Libro no encontrado')
    return item

def get_or_create_relation(user_id: int, book_id: int):
    res = supabase.table('tbl_libros_x_usuarios').select('id_libro_usuario').eq('id_usuario', user_id).eq('id_libro', book_id).maybe_single().execute()
    item = getattr(res, 'data', None)
    if item:
        return item['id_libro_usuario']
    now = datetime.utcnow().isoformat()
    ins = supabase.table('tbl_libros_x_usuarios').insert({
        'id_usuario': user_id, 'id_libro': book_id, 'id_estado': 3,
        'progreso': 0, 'pagina': 0, 'palabra': 0, 'tiempo_escucha': 0,
        'fecha_ultima_lectura': now
    }).select('id_libro_usuario').maybe_single().execute()
    it = getattr(ins, 'data', None)
    return it['id_libro_usuario'] if it else None

def update_progress(user_id: int, book_id: int, audio_url: str, voice_id: int, rate: float):
    now = datetime.utcnow().isoformat()
    supabase.table('tbl_libros_x_usuarios').update({
        'audio': audio_url, 'id_voz': voice_id, 'id_playbackrate': rate, 'fecha_ultima_lectura': now
    }).eq('id_usuario', user_id).eq('id_libro', book_id).execute()
    print('✅ Progreso actualizado en tbl_libros_x_usuarios')

print('✅ Helpers cargados')

In [ ]:
# Descarga del PDF
voice_short = get_voice_short_name(VOICE_ID)
book = get_book(BOOK_ID)
archivo = book.get('archivo')
print('🎙️ Voz seleccionada:', voice_short)
print('📘 Libro:', book.get('titulo'), '| archivo =', archivo)

def download_pdf_bytes(archivo: str) -> bytes:
    if archivo.startswith('http://') or archivo.startswith('https://'):
        b, p = resolve_bucket_and_path_from_url(archivo)
        if b and p:
            data = supabase.storage.from_(b).download(p)
            return data
        resp = requests.get(archivo)
        resp.raise_for_status()
        return resp.content
    else:
        return supabase.storage.from_(BUCKET_ARCHIVOS).download(archivo)

pdf_bytes = download_pdf_bytes(archivo)
print('⬇️ PDF descargado, bytes =', len(pdf_bytes))

In [ ]:
# Extracción de texto: PyPDF2 -> fallback pdfminer.six
def strip_leading_page_token(line: str) -> str:
    return re.sub(r'^\s*(?:\d{1,4}|[ivxlcdmIVXLCDM]{1,7})[)\].,:;—\-–]?\s+', '', line or '')

def is_page_number(line: str) -> bool:
    s = (line or '').strip()
    return s.isdigit() or (re.fullmatch(r'[ivxlcdmIVXLCDM]+', s or '') is not None and len(s) <= 5)

HEADERS = {
    'Cien años de soledad', 'Gabriel  García Márquez', 'Gabriel García Márquez'
}
def is_header_footer(line: str) -> bool:
    return (line or '').strip() in HEADERS

def merge_lines_into_paragraphs(lines):
    cleaned, collapsed, paragraphs = [], [], []
    for raw in lines:
        line = (raw or '').replace('­', '')
        line = strip_leading_page_token(line)
        line = re.sub(r'\s+', ' ', line).rstrip()
        if not line:
            cleaned.append('')
            continue
        if is_page_number(line) or is_header_footer(line):
            continue
        cleaned.append(line)
    blank = False
    for l in cleaned:
        if l == '':
            if not blank:
                collapsed.append('')
                blank = True
        else:
            collapsed.append(l)
            blank = False
    current = ''
    for l in collapsed:
        if l == '':
            if current:
                paragraphs.append(current.strip())
                current = ''
            else:
                paragraphs.append('')
            continue
        if current == '':
            current = l.strip()
        else:
            if current.endswith('-'):
                current = current[:-1] + l.lstrip()
            else:
                current += ' ' + l.lstrip()
    if current:
        paragraphs.append(current.strip())
    return '

'.join(paragraphs).strip()

def extract_text_with_pypdf2(pdf_bytes: bytes) -> str:
    try:
        r = PdfReader(BytesIO(pdf_bytes))
        all_lines = []
        for page in r.pages:
            t = page.extract_text() or ''
            if not t: continue
            all_lines.extend(t.splitlines()); all_lines.append('')
        return merge_lines_into_paragraphs(all_lines)
    except Exception:
        return ''

def extract_text_with_pdfminer(pdf_bytes: bytes) -> str:
    try:
        return (pdf_extract_text(BytesIO(pdf_bytes)) or '').strip()
    except Exception:
        return ''

print('🔎 Extrayendo texto con PyPDF2...')
text = extract_text_with_pypdf2(pdf_bytes)
print('  -> caracteres PyPDF2 =', len(text))
if not text or len(text) < 50:
    print('⚠️ PyPDF2 insuficiente; probando pdfminer.six...')
    text = extract_text_with_pdfminer(pdf_bytes)
    print('  -> caracteres pdfminer =', len(text))
if not text or len(text) < 50:
    raise RuntimeError('No se pudo extraer texto suficiente del PDF')
print('✅ Texto extraído OK')

# Guardar .txt junto al PDF en el mismo bucket de archivos
b, p = resolve_bucket_and_path_from_url(archivo)
if not (b and p):
    b, p = BUCKET_ARCHIVOS, archivo
txt_path = p.rsplit('.', 1)[0] + '.txt'
try:
    supabase.storage.from_(b).upload(txt_path, text.encode('utf-8'), { 'contentType': 'text/plain', 'upsert': 'true' })
    print('💾 Texto guardado en', f'{b}/{txt_path}')
except Exception as e:
    print('⚠️ No se pudo guardar .txt:', e)

In [ ]:
# División del texto en fragmentos
def chunk_text(texto: str, max_chars: int = MAX_CHARS):
    texto = re.sub(r'\s+', ' ', texto or '').strip()
    if len(texto) <= max_chars:
        return [texto]
    oraciones = re.split(r'(?<=[\.!?
])\s+', texto)
    chunks, actual = [], ''
    for oracion in oraciones:
        if len(actual) + len(oracion) + 1 <= max_chars:
            actual = (actual + ' ' + oracion).strip()
        else:
            if actual:
                chunks.append(actual)
            while len(oracion) > max_chars:
                chunks.append(oracion[:max_chars])
                oracion = oracion[max_chars:]
            actual = oracion
    if actual:
        chunks.append(actual)
    return chunks

chunks = chunk_text(text, MAX_CHARS)
print(f'✂️ Texto dividido en {len(chunks)} fragmentos de hasta {MAX_CHARS} caracteres.')

In [ ]:
# Síntesis TTS por fragmentos -> bytes MP3
def rate_to_pct(rate: float) -> str:
    pct = int((rate - 1.0) * 100)
    s = f'{pct}%'
    if not s.startswith('-') and not s.startswith('+'):
        s = '+' + s
    return s

async def synthesize_to_mp3_bytes(chunks, voice_short_name: str, rate: float = 1.0) -> bytes:
    out = bytearray()
    total = len(chunks)
    rate_pct = rate_to_pct(rate)
    for i, texto in enumerate(chunks, start=1):
        comunicador = edge_tts.Communicate(text=texto, voice=voice_short_name, rate=rate_pct)
        async for evt in comunicador.stream():
            if evt['type'] == 'audio':
                out.extend(evt['data'])
        print(f'✅ Fragmento {i}/{total} sintetizado')
    return bytes(out)

# Ejecutar síntesis manejando event loop en Notebook
async def _run_synth():
    return await synthesize_to_mp3_bytes(chunks, voice_short, RATE)

try:
    loop = asyncio.get_event_loop()
    if loop.is_running():
        audio_bytes = await _run_synth()
    else:
        audio_bytes = asyncio.run(_run_synth())
except RuntimeError:
    audio_bytes = await _run_synth()

print('🎧 MP3 generado, bytes =', len(audio_bytes))
if len(audio_bytes) < 50000:
    raise RuntimeError('Audio demasiado corto; revisa el texto extraído')

In [ ]:
# Subida a Storage y actualización de la relación
dest_dir = str(BOOK_ID)
dest_name = f'{voice_short}.mp3'
dest_path = f'{dest_dir}/{dest_name}'
print('📤 Subiendo MP3 a', f'{BUCKET_AUDIOS}/{dest_path}')
supabase.storage.from_(BUCKET_AUDIOS).upload(dest_path, audio_bytes, { 'contentType': 'audio/mpeg', 'upsert': 'true' })
pub = supabase.storage.from_(BUCKET_AUDIOS).get_public_url(dest_path)
audio_url = pub.get('publicUrl') if isinstance(pub, dict) else pub['publicUrl']
print('🔗 Audio URL pública:', audio_url)

# Asegurar relación y actualizar progreso
get_or_create_relation(USER_ID, BOOK_ID)
update_progress(USER_ID, BOOK_ID, audio_url, VOICE_ID, RATE)
print('🏁 Proceso completo')